In [249]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
os.getcwd()

'C:\\Users\\Shrey'

In [3]:
#Change directory to where dataset is
os.chdir("X:\\Datasets")

In [4]:
transaction_data=pd.read_excel('Fraud_Analytics_Dataset.xlsx',0)
customer_data=pd.read_excel('Fraud_Analytics_Dataset.xlsx',1)
sales_data=pd.read_excel('Fraud_Analytics_Dataset.xlsx',2)

In [5]:
display(transaction_data.head())
display(customer_data.head())
display(sales_data.head())

,Transaction_Id,First Time Customers.,Bigger than Average Orders.,Fast Shipping,Unusual Location.,Large quantity of Same Product .,Multiple Shipping Addresses.,Shipping and Billing Address are not the same.,Several Cards used from the Same Ip Address.,Payment Information typed with Capital letters.,Many Transactions in a short time period.
0,235360,No,No,No,No,No,No,No,No,No,No
1,234773,No,No,No,No,No,No,No,No,No,No
2,235242,No,No,No,No,No,No,No,No,No,No
3,235350,No,Yes,Yes,No,No,Yes,Yes,Yes,Yes,Yes
4,234914,No,No,No,No,No,No,No,No,No,No


,CUSTOMER_ID,LOYALTY_ID,MEMBERSHIP_TYPE__ID,MEMBERSHIP_TYPE,MEMBERSHIP_TYPE_SC,CUSTOMER_FLG,Customer_Type,customer_type_id,customer_type_sc,behavior_segment,...,BIRTH_MONTH,BIRTH_YEAR,BIRTH_QUARTER,PLACE_OF_BIRTH,COUNTRY_OF_BIRTH_SC,EVER_BANKRUPT_FLG,NO_TIMES_DELINQUENT_in_365_days,BLACKLISTED_FLG,NO_OF_DEPENDANTS,Record_created_By
0,76565665,LID877073,901.0,Gold,GLD,Y,Small/Medium Enterprise,1002.0,ENT,High Value,...,April,2004,2,Bellevue,US,N,1,N,3,\r
1,76565666,LID877029,NaN,NaN,NaN,Y,Individual,1003.0,INDV,Low Value,...,December,1984,4,Jackson,US,N,1,N,1,\r
2,76565667,LID876876,902.0,Silver,SLV,Y,Small/Medium Enterprise,1002.0,ENT,Medium Value,...,February,1999,1,Chattanooga,US,N,1,N,3,\r
3,76565668,LID876841,902.0,Silver,SLV,Y,Small/Medium Enterprise,1002.0,ENT,Low Value,...,December,1997,4,Worcester,US,N,0,N,1,\r
4,76565669,LID877001,902.0,Silver,SLV,Y,Individual,1003.0,INDV,Medium Value,...,January,1992,1,Los Angeles,US,N,1,N,1,\r


,Transaction_Id,Invoice_Id,Order_Id,Customer_Id,Mdm_Customer_Id,Loyalty_Id,Gender,Age,Membership_Id,Membership_Type,...,Cancel_Reason_Code,Cancel_Description,Return_Flag,Return_Dt,Return_Amt,Order_Returnpaid_Gross_Amt,Lastmodified_Dt,Sales_Rep_Name,Sales_Rep_Id,Fraud
0,234825,88371,88371,76565900,954421840,LID876561,FEMALE,26,904.0,Platinum,...,0,NaN,Y,2016-05-08,1953,1406,2016-04-01,NaN,\r,No
1,234666,88234,88234,76566122,965982087,NaN,MALE,22,NaN,NaN,...,0,NaN,N,2016-04-26,0,0,2016-11-14,NaN,\r,Yes
2,234823,88153,88153,76565906,953091566,LID876543,MALE,19,904.0,Platinum,...,0,NaN,N,2016-05-16,0,0,2016-08-25,John,S101\r,No
3,235342,87867,87867,76566458,924245622,LID876557,MALE,26,901.0,Gold,...,0,NaN,N,2016-05-14,0,0,2016-04-17,NaN,\r,No
4,234739,87945,87945,76566387,966653489,LID876640,MALE,37,901.0,Gold,...,0,NaN,Y,2016-05-26,1322,952,2016-07-04,John,S101\r,Yes


In [99]:
'''
Joining all the 3 sheets
'''
#caller.join(other.set_index('key'), on='key')
f1=sales_data.join(transaction_data.set_index('Transaction_Id'),on='Transaction_Id')
original_data = f1.join(customer_data.set_index('CUSTOMER_ID'),on='Customer_Id')

In [100]:
#Preprocessing - > Wrangling + Cleaning

# Correcting column names for ease of usage
original_data.columns = original_data.columns.str.replace(' ','')
original_data.columns = original_data.columns.str.replace('.','')

#Making object type data into categorical variables for now 
# for col_name in master_data.columns:
#     if(master_data[col_name].dtype == 'object'):
#         master_data[col_name]= master_data[col_name].astype('category')
#         #master_data[col_name] = transaction_data[col_name].cat.codes
        


In [272]:
#Please don't touch this piece of code, otherwise the destruction caused would be equivalent to World War 3
master_data = original_data

In [273]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Columns: 157 entries, Transaction_Id to Record_created_By
dtypes: datetime64[ns](3), float64(13), int64(39), int8(10), object(92)
memory usage: 1.1+ MB


---
Through a basic Excel Sheet analysis, we have are seeing a lot of redundant variables as well as duplicate ones. 

1. Columns representing unique values such as Transaction ID's or Customer ID's can be removed directly.

2. Multiple variables such as Loyalty ID, Membership related Info, etc. are representing the same information, leading to **Multicollinearity in the data**. Such variables can be romoved or clubbed together to create a single informative column.

3. Some variables, which are not relevant within the aim of this project, such as Country, can be removed, as they contains only a single value. If and when this data is extrapolated for other countries, it may prove to be useful.

4. Fields with large number of missing values, as we can't assume the values in such cases for better chances of accuracy.

We will not be using such columns, and a single list of columns being used for model building will be informed.

---

In [274]:
feature_list = ['No_Of_Items','Shipment_Flg','Return_Flag','FirstTimeCustomers',
                'UnusualLocation','BiggerthanAverageOrders','MultipleShippingAddresses','ShippingandBillingAddressarenotthesame',
                'SeveralCardsusedfromtheSameIpAddress','PaymentInformationtypedwithCapitalletters', 'FastShipping',
                'ManyTransactionsinashorttimeperiod','MEMBERSHIP_TYPE__ID','CUSTOMER_STATUS_ID','customer_type_id','AGE',
                'INCOME_RANGE', 'PROFESSION', 'Preferred_Comm_Sub_Channel','credit_risk_rating', 'Residence_Province', 
                'Mode_Of_Payment','Sale_Dt']
target = 'Fraud'

In [275]:
label_data = master_data[target]
master_data = master_data[feature_list]
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 23 columns):
No_Of_Items                                  947 non-null int64
Shipment_Flg                                 947 non-null object
Return_Flag                                  947 non-null object
FirstTimeCustomers                           947 non-null int8
UnusualLocation                              947 non-null int8
BiggerthanAverageOrders                      947 non-null int8
MultipleShippingAddresses                    947 non-null int8
ShippingandBillingAddressarenotthesame       947 non-null int8
SeveralCardsusedfromtheSameIpAddress         947 non-null int8
PaymentInformationtypedwithCapitalletters    947 non-null int8
FastShipping                                 947 non-null int8
ManyTransactionsinashorttimeperiod           947 non-null int8
MEMBERSHIP_TYPE__ID                          674 non-null float64
CUSTOMER_STATUS_ID                           921 non-null float64
cu

In [276]:
obj_type_variables = [column for column in master_data.columns if master_data[column].dtype == 'object']
print(obj_type_variables)

['Shipment_Flg', 'Return_Flag', 'INCOME_RANGE', 'PROFESSION', 'Preferred_Comm_Sub_Channel', 'credit_risk_rating', 'Residence_Province', 'Mode_Of_Payment', 'Sale_Dt']


In [279]:
# Feature Engineering
master_data["day_of_week"] = master_data['Sale_Dt'].apply(lambda x:pd.to_datetime(x).dayofweek)
master_data.drop(columns = ['Sale_Dt'])
master_data.head()

C:\Users\Shrey\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,No_Of_Items,Shipment_Flg,Return_Flag,FirstTimeCustomers,UnusualLocation,BiggerthanAverageOrders,MultipleShippingAddresses,ShippingandBillingAddressarenotthesame,SeveralCardsusedfromtheSameIpAddress,PaymentInformationtypedwithCapitalletters,...,customer_type_id,AGE,INCOME_RANGE,PROFESSION,Preferred_Comm_Sub_Channel,credit_risk_rating,Residence_Province,Mode_Of_Payment,Sale_Dt,day_of_week
0,7,Y,Y,0,0,0,0,0,0,0,...,1003.0,34,60K-90K,Employed,IVR,High,FL,Credit Card,2016-04-08,4
1,10,Y,N,0,0,0,0,0,0,0,...,1003.0,34,<30K,Employed,Web,Low,NJ,Debit Card,2016-03-27,6
2,2,N,N,0,1,1,1,1,1,1,...,1002.0,32,>90K,Employed,IVR,Low,AK,Cash,2016-04-16,5
3,16,N,N,0,0,0,0,0,0,0,...,1002.0,28,>90K,Employed,Call,Medium,AK,Credit Card,2016-04-14,3
4,3,Y,Y,0,0,0,0,0,0,0,...,1002.0,41,>90K,Business,IVR,Medium,SC,Debit Card,2016-04-26,1


In [295]:
#Count according to day
day_dict = dict()
for val in master_data.day_of_week:
    if val not in day_dict.keys():
        day_dict[val] = 1
    else:
        day_dict[val] += 1
print(day_dict)

{4: 137, 6: 126, 5: 142, 3: 130, 1: 167, 2: 122, 0: 123}


In [280]:
#Making object type data into categorical variables for now
for column in obj_type_variables:
    master_data[column] = master_data[column].astype('category')

C:\Users\Shrey\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [281]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 24 columns):
No_Of_Items                                  947 non-null int64
Shipment_Flg                                 947 non-null category
Return_Flag                                  947 non-null category
FirstTimeCustomers                           947 non-null int8
UnusualLocation                              947 non-null int8
BiggerthanAverageOrders                      947 non-null int8
MultipleShippingAddresses                    947 non-null int8
ShippingandBillingAddressarenotthesame       947 non-null int8
SeveralCardsusedfromtheSameIpAddress         947 non-null int8
PaymentInformationtypedwithCapitalletters    947 non-null int8
FastShipping                                 947 non-null int8
ManyTransactionsinashorttimeperiod           947 non-null int8
MEMBERSHIP_TYPE__ID                          674 non-null float64
CUSTOMER_STATUS_ID                           921 non-null float6

In [282]:
#Converting categorical into numerical values
master_data[obj_type_variables] = master_data[obj_type_variables].apply(lambda column: column.cat.codes)

C:\Users\Shrey\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [283]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 24 columns):
No_Of_Items                                  947 non-null int64
Shipment_Flg                                 947 non-null int8
Return_Flag                                  947 non-null int8
FirstTimeCustomers                           947 non-null int8
UnusualLocation                              947 non-null int8
BiggerthanAverageOrders                      947 non-null int8
MultipleShippingAddresses                    947 non-null int8
ShippingandBillingAddressarenotthesame       947 non-null int8
SeveralCardsusedfromtheSameIpAddress         947 non-null int8
PaymentInformationtypedwithCapitalletters    947 non-null int8
FastShipping                                 947 non-null int8
ManyTransactionsinashorttimeperiod           947 non-null int8
MEMBERSHIP_TYPE__ID                          674 non-null float64
CUSTOMER_STATUS_ID                           921 non-null float64
custom

In [284]:
#list of columns with null values 
list_of_clm_null_value= master_data.columns[master_data.isna().any()].tolist()
print(list_of_clm_null_value)

['MEMBERSHIP_TYPE__ID', 'CUSTOMER_STATUS_ID', 'customer_type_id']


In [285]:
#replacing the nan value with -1 after EDA
for i in list_of_clm_null_value:
    master_data[i].fillna(-1, inplace = True) 

C:\Users\Shrey\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [286]:
#Converting numerical based categories into category dtype
for column in obj_type_variables:
    master_data[column] = master_data[column].astype('category')

C:\Users\Shrey\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


---
Model Building
---
---

In [287]:
#Stratified Shuffle Split by 70:30 ratio

retail_data = master_data
features_train, features_test, labels_train, labels_test = train_test_split(retail_data, label_data, test_size=0.3, random_state=42)

In [288]:
features_train.columns

Index(['No_Of_Items', 'Shipment_Flg', 'Return_Flag', 'FirstTimeCustomers',
       'UnusualLocation', 'BiggerthanAverageOrders',
       'MultipleShippingAddresses', 'ShippingandBillingAddressarenotthesame',
       'SeveralCardsusedfromtheSameIpAddress',
       'PaymentInformationtypedwithCapitalletters', 'FastShipping',
       'ManyTransactionsinashorttimeperiod', 'MEMBERSHIP_TYPE__ID',
       'CUSTOMER_STATUS_ID', 'customer_type_id', 'AGE', 'INCOME_RANGE',
       'PROFESSION', 'Preferred_Comm_Sub_Channel', 'credit_risk_rating',
       'Residence_Province', 'Mode_Of_Payment', 'Sale_Dt', 'day_of_week'],
      dtype='object')

In [289]:
gnb = GaussianNB()
y_pred_gnb = gnb.fit(features_train, labels_train).predict(features_test)

DT = DecisionTreeClassifier()
y_pred_DT = DT.fit(features_train, labels_train).predict(features_test)

ada = AdaBoostClassifier()
y_pred_ada = ada.fit(features_train, labels_train).predict(features_test)

In [290]:
print("GNB:", accuracy_score(labels_test, y_pred_gnb))
print("DT:", accuracy_score(labels_test, y_pred_DT))
print("ADA:", accuracy_score(labels_test, y_pred_ada))



# #print precision_recall_fscore_support(predicted_variable, y_pred, average='macro')
# print(precision_recall_fscore_support(predicted_variable, y_pred, average='micro'))
# print(precision_recall_fscore_support(predicted_variable, y_pred, average=None))
# #print precision_recall_fscore_support(predicted_variable, y_pred, average='weighted')

# print(math.sqrt(mean_squared_error(predicted_variable, y_pred)))


GNB: 0.8035087719298246
DT: 0.7298245614035088
ADA: 0.8210526315789474


In [291]:
print("GNB:", classification_report(labels_test, y_pred_gnb))
print("DT:", classification_report(labels_test, y_pred_DT))
print("ADA:", classification_report(labels_test, y_pred_ada))

GNB:              precision    recall  f1-score   support

         No       0.86      0.92      0.89       241
        Yes       0.27      0.16      0.20        44

avg / total       0.77      0.80      0.78       285

DT:              precision    recall  f1-score   support

         No       0.84      0.85      0.84       241
        Yes       0.10      0.09      0.09        44

avg / total       0.72      0.73      0.73       285

ADA:              precision    recall  f1-score   support

         No       0.84      0.97      0.90       241
        Yes       0.00      0.00      0.00        44

avg / total       0.71      0.82      0.76       285

